In [25]:
import cellCnn
import importlib

importlib.reload(cellCnn)
import random
import numpy as np
### (from: https://github.com/eiriniar/CellCnn/blob/0413a9f49fe0831c8fe3280957fb341f9e028d2d/cellCnn/examples/NK_cell_ungated.ipynb ) AND https://github.com/eiriniar/CellCnn/blob/0413a9f49fe0831c8fe3280957fb341f9e028d2d/cellCnn/examples/PBMC.ipynb
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
import seaborn as sns
import matplotlib.pyplot as plt
from cellCnn.ms.utils.helpers import get_chunks
from cellCnn.ms.utils.helpers import print_regression_model_stats
from cellCnn.plotting import plot_results
from cellCnn.utils import mkdir_p
from cellCnn.utils import save_results
from cellCnn.ms.utils.helpers import get_fitted_model
from cellCnn.ms.utils.helpers import split_test_valid_train
from cellCnn.ms.utils.helpers import calc_frequencies, get_chunks_from_df
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score


In [26]:
#%reset

In [27]:
##### state vars
cytokines = ['CCR2', 'CCR4', 'CCR6', 'CCR7', 'CXCR4', 'CXCR5', 'CD103', 'CD14', 'CD20', 'CD25', 'CD27', 'CD28', 'CD3',
             'CD4', 'CD45RA', 'CD45RO', 'CD56', 'CD57', 'CD69', 'CD8', 'TCRgd', 'PD.1', 'GM.CSF', 'IFN.g', 'IL.10',
             'IL.13', 'IL.17A', 'IL.2', 'IL.21', 'IL.22', 'IL.3', 'IL.4', 'IL.6', 'IL.9', 'TNF.a']
infile = 'cohort_denoised_clustered_diagnosis_patients.csv'
indir = 'data/input'
outdir = 'out_ms_default'
rand_seed = 123
train_perc = 0.7
test_perc = 0.3
batch_size_pheno = 840 # deprecated  # so a size of 8425 is about equally sized in batches
batch_size_cd4 = 550  # deprecated #so a size of 550 gets me 16 batches for cd4
## information from ms_data project
cluster_to_celltype_dict = {0: 'b', 1: 'cd4', 3: 'nkt', 8: 'cd8', 10: 'nk', 11: 'my', 16: 'dg'}

In [28]:
np.random.seed(rand_seed)
mkdir_p(outdir)
df = pd.read_csv(os.path.join(indir, infile), index_col=0)
df = df.drop_duplicates()  ### reduces overfitting at cost of fewer data
df.shape
##### no duplicates in

(16889, 38)

In [29]:
plt.figure()
patients_clusters_conf_table = pd.crosstab(df['gate_source'], df['cluster'])
sns.heatmap(patients_clusters_conf_table, annot=False, vmax=100)
plt.show()
#plt.savefig('images/patient_vs_cluster_conf_table.png')
patients_clusters_conf_table

/var/folders/p1/5d38cf1d6tvdyny38zb_66540000gn/T/ipykernel_37221/3510062378.py:4: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


cluster,0,1,3,8,10,11,16
gate_source,,,,,,,
3.0,60,119,6,30,5,5,0
4.0,15,46,0,45,2,1,13
5.0,33,131,0,70,2,2,4
8.0,24,122,0,31,10,11,4
9.0,72,270,2,137,11,10,4
...,...,...,...,...,...,...,...
128.0,4,6,4,6,1,6,0
130.0,24,82,2,56,8,4,0
131.0,20,48,2,14,13,7,0


In [30]:
# pitch: key = gate_source, value = dict{diagnosis: df OR freq?}
rrms_df = df[df['diagnosis'] == 'RRMS']
rrms_patients2df = {id: patient_df.drop(columns=['diagnosis', 'gate_source']) for id, patient_df in
                    rrms_df.groupby('gate_source')}
nindc_df = df[df['diagnosis'] == 'NINDC']
nindc_patients2df = {id: patient_df.drop(columns=['diagnosis', 'gate_source']) for id, patient_df in
                     nindc_df.groupby('gate_source')}

In [31]:
print('RRMS cell-type abundances')
rrms_df.groupby('cluster').count()

RRMS cell-type abundances


,CD69,CXCR5,CCR2,CD57,CD28,IL.9,IL.4,CD27,IL.3,IL.13,...,CD25,CD4,CD3,IL.21,IL.17A,CXCR4,CCR4,CD14,diagnosis,gate_source
cluster,,,,,,,,,,,,,,,,,,,,,
0,1114,1114,1114,1114,1114,1114,1114,1114,1114,1114,...,1114,1114,1114,1114,1114,1114,1114,1114,1114,1114
1,4302,4302,4302,4302,4302,4302,4302,4302,4302,4302,...,4302,4302,4302,4302,4302,4302,4302,4302,4302,4302
3,147,147,147,147,147,147,147,147,147,147,...,147,147,147,147,147,147,147,147,147,147
8,2113,2113,2113,2113,2113,2113,2113,2113,2113,2113,...,2113,2113,2113,2113,2113,2113,2113,2113,2113,2113
10,401,401,401,401,401,401,401,401,401,401,...,401,401,401,401,401,401,401,401,401,401
11,216,216,216,216,216,216,216,216,216,216,...,216,216,216,216,216,216,216,216,216,216
16,171,171,171,171,171,171,171,171,171,171,...,171,171,171,171,171,171,171,171,171,171


In [32]:
print('NINDC cell-type abundances')
nindc_df.groupby('cluster').count()

NINDC cell-type abundances


,CD69,CXCR5,CCR2,CD57,CD28,IL.9,IL.4,CD27,IL.3,IL.13,...,CD25,CD4,CD3,IL.21,IL.17A,CXCR4,CCR4,CD14,diagnosis,gate_source
cluster,,,,,,,,,,,,,,,,,,,,,
0,1141,1141,1141,1141,1141,1141,1141,1141,1141,1141,...,1141,1141,1141,1141,1141,1141,1141,1141,1141,1141
1,4501,4501,4501,4501,4501,4501,4501,4501,4501,4501,...,4501,4501,4501,4501,4501,4501,4501,4501,4501,4501
3,219,219,219,219,219,219,219,219,219,219,...,219,219,219,219,219,219,219,219,219,219
8,1858,1858,1858,1858,1858,1858,1858,1858,1858,1858,...,1858,1858,1858,1858,1858,1858,1858,1858,1858,1858
10,379,379,379,379,379,379,379,379,379,379,...,379,379,379,379,379,379,379,379,379,379
11,202,202,202,202,202,202,202,202,202,202,...,202,202,202,202,202,202,202,202,202,202
16,125,125,125,125,125,125,125,125,125,125,...,125,125,125,125,125,125,125,125,125,125


In [33]:
importlib.reload(cellCnn.ms.utils.helpers)
from cellCnn.ms.utils.helpers import *

#### here we could see freq differences across the 2 groups
print('Frequencies: ')
rrms_patients_freq = {id: calc_frequencies(patient_df, cluster_to_celltype_dict, return_list=True) for id, patient_df in
                      rrms_patients2df.items()}
nindc_patients_freq = {id: calc_frequencies(patient_df, cluster_to_celltype_dict, return_list=True) for id, patient_df
                       in nindc_patients2df.items()}
print('DONE')
# we got 31 patients each

Frequencies: 
DONE


In [34]:
#### To get true frequencies we need to get rid of 0 entries (there are patient without some cells type due to data)
print('RRMS')
rrms_freq_df = pd.DataFrame(list(rrms_patients_freq.values()), columns=list(cluster_to_celltype_dict.values()))
rrms_freq_df = rrms_freq_df.replace(0, np.NaN)  ## this lets us skip the 0 entries
rrms_freq_df.describe()

RRMS


,b,cd4,nkt,cd8,nk,my,dg
count,31.000000,31.000000,28.000000,31.000000,31.000000,31.000000,25.000000
mean,0.145822,0.470019,0.034702,0.242621,0.047500,0.038641,0.029826
std,0.062796,0.127876,0.043881,0.068406,0.031251,0.041190,0.026293
min,0.044118,0.222222,0.003953,0.120543,0.008264,0.004662,0.001832
25%,0.098750,0.393336,0.008248,0.193943,0.022284,0.017099,0.011655
50%,0.136364,0.461538,0.018236,0.243386,0.045455,0.023810,0.019802
75%,0.190358,0.543535,0.032926,0.283517,0.057529,0.044735,0.042857
max,0.285714,0.738540,0.178571,0.349057,0.125000,0.222222,0.128205


In [35]:
print('NINDC')
nindc_freq_df = pd.DataFrame(list(nindc_patients_freq.values()), columns=list(cluster_to_celltype_dict.values()))
nindc_freq_df = nindc_freq_df.replace(0, np.NaN)
nindc_freq_df.describe()

NINDC


,b,cd4,nkt,cd8,nk,my,dg
count,31.000000,31.000000,30.000000,31.000000,30.000000,29.000000,26.000000
mean,0.160800,0.440072,0.043986,0.245588,0.059403,0.039039,0.020230
std,0.107951,0.155058,0.038204,0.094339,0.047733,0.033544,0.020225
min,0.033149,0.189189,0.001524,0.046559,0.011236,0.001706,0.002841
25%,0.076150,0.329932,0.012810,0.206028,0.024907,0.014587,0.007189
50%,0.142857,0.390476,0.039086,0.247573,0.037789,0.031250,0.017101
75%,0.215665,0.516367,0.067342,0.290557,0.086959,0.050000,0.024923
max,0.433198,0.784572,0.142857,0.410061,0.200000,0.142857,0.106557


In [36]:
batch_sizes = [40, 80, 100,150,200, 250]
batch_size_dict = dict()
#batch_size = 20
cluster = 1
for batch_size in batch_sizes:
    ### desease states 1 = RRMS and 0 = NINDC
    selection_pool_rrms_cd4, too_few_data_rrms_cd4 = get_chunks_from_df(rrms_patients2df,
                           freq_df=rrms_patients_freq,
                           desease_state=1,
                           cluster=cluster,
                           batch_size=batch_size)
    selection_pool_nindc_cd4, too_few_data_nindc_cd4 = get_chunks_from_df(nindc_patients2df,
                           freq_df=nindc_patients_freq,
                           desease_state=0,
                           cluster=cluster,
                           batch_size=batch_size)
    #todo make sure list are equally long:
    if len(selection_pool_rrms_cd4) > len(selection_pool_nindc_cd4):
        selection_pool_rrms_cd4 = selection_pool_rrms_cd4[:len(selection_pool_nindc_cd4)]
    elif len(selection_pool_rrms_cd4) < len(selection_pool_nindc_cd4):
        selection_pool_nindc_cd4 = selection_pool_nindc_cd4[:len(selection_pool_rrms_cd4)]

    all_chunks = selection_pool_rrms_cd4 + selection_pool_nindc_cd4
    np.random.shuffle(all_chunks)

    X = [selection[0] for selection in all_chunks]
    cd4 = [selection[1] for selection in all_chunks]
    Y = [selection[2] for selection in all_chunks]

    batch_size_dict[batch_size] = (X, cd4, Y)

print('prep done')

prep done


In [27]:
importlib.reload(cellCnn.ms.utils.helpers)
importlib.reload(cellCnn.model)
from cellCnn.ms.utils.helpers import *
from cellCnn.model import CellCnn

outdir_pheno = 'ms_pheno_class_cd4_chunks_all'
stats_dict_class = dict()
for batch_size, values in batch_size_dict.items():
    kf = StratifiedKFold(n_splits=3, random_state=rand_seed)
    model_container = []
    X, Y = values[0], values[2]
    X_test, X_train, X_valid, y_test, y_train, y_valid = split_test_valid_train(X=X,
        y=Y,
        test_perc=test_perc,
        train_perc=train_perc,
        valid_perc=0.5, seed=rand_seed)
    X = np.array([*X_train, *X_valid])
    y = np.array([*y_train, *y_valid])
    for train_idx, valid_idx, in kf.split(X=X, y=y):
        X_train, X_valid = X[train_idx], X[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]
        model = get_fitted_model(X_train, X_valid, y_train, y_valid,
                                 nsubset=int(batch_size*len(X_train)/2),
                                 nfilters=[3, 15, 25, 35], coeff_l1=0,
                                 max_epochs=100, nrun=30, learning_rate=None,
                                 ncell=batch_size,
                                 per_sample=False, regression=False,
                                 outdir=outdir_pheno, verbose=False)
        model_container.append(model)
        results = model.results
    accs = []
    rocs = []
    for model in model_container:
        print('BS:' + str(batch_size))
        results = model.results
        test_pred = model.predict(X_test)
        #print_regression_model_stats(test_pred, y_test)
        test_pred_abs = [1 if pred[0]>pred[1] else 0 for pred in test_pred ]
        acc = accuracy_score(y_test, test_pred_abs)
        print(f'Acc : {acc}')
        accs.append(acc)
    acc = sum(accs)/len(accs)
    print(f'For: {batch_size}:')
    print(f'Mean Acc: {str(acc)}')
    stats_dict_class[batch_size] = {'mean_acc': acc}

print('DONE')

/Users/eliasschreiner/.local/share/virtualenvs/CellCnn-O6mYCfgp/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


47/47 [==============================] - 0s 2ms/step - loss: 0.4467 - accuracy: 0.8331
BS:100
Acc : 0.75
BS:100
Acc : 0.5
BS:100
Acc : 0.5
For: 100:
Mean Acc: 0.5833333333333334


/Users/eliasschreiner/.local/share/virtualenvs/CellCnn-O6mYCfgp/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


57/57 [==============================] - 0s 1ms/step - loss: 0.6653 - accuracy: 0.7356
BS:150
Acc : 0.6666666666666666
BS:150
Acc : 0.6666666666666666
BS:150
Acc : 0.3333333333333333
For: 150:
Mean Acc: 0.5555555555555555


/Users/eliasschreiner/.local/share/virtualenvs/CellCnn-O6mYCfgp/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


50/50 [==============================] - 0s 2ms/step - loss: 0.6182 - accuracy: 0.5656
BS:200
Acc : 0.6666666666666666
BS:200
Acc : 0.6666666666666666
BS:200
Acc : 0.6666666666666666
For: 200:
Mean Acc: 0.6666666666666666


/Users/eliasschreiner/.local/share/virtualenvs/CellCnn-O6mYCfgp/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


47/47 [==============================] - 0s 4ms/step - loss: 0.6972 - accuracy: 0.5000
BS:250
Acc : 0.5
BS:250
Acc : 1.0
BS:250
Acc : 0.5
For: 250:
Mean Acc: 0.6666666666666666
DONE


In [28]:
#### NOTE:
#### it performs Strat.KFold anyways (even if i dont manually divide them into train valids... )
#### bad thing is that i dont have any test yet
#### made Errors:
#### 1. I put in y_unique as the true class labels (no integer in a range) and got a list indes out of bound
print('######################################')
print('######################################')
print(' CLASSIFICATION task DONE')
print('######################################')
print('######################################')


classification task DONE


In [131]:
# plot the results of the CellCnn analysis for the test samples in the output directory
_ = plot_results(model.results, X_test, y_test,
                 cytokines, outdir + '/plots', filter_response_thres=0,
                 filter_diff_thres=0.2, group_a='RRMS', group_b='NINDC')
print('done')
### plotting stuff

done


In [35]:
batch_size = 200
cluster = 1
### desease states 1 = RRMS and 0 = NINDC
selection_pool_rrms_cd4, too_few_data_rrms_cd4 = get_chunks_from_df(rrms_patients2df,
                       freq_df=rrms_patients_freq,
                       desease_state=1,
                       cluster=cluster,
                       batch_size=batch_size)
selection_pool_nindc_cd4, too_few_data_nindc_cd4 = get_chunks_from_df(nindc_patients2df,
                       freq_df=nindc_patients_freq,
                       desease_state=0,
                       cluster=cluster,
                       batch_size=batch_size)
#todo make sure list are equally long:
if len(selection_pool_rrms_cd4) > len(selection_pool_nindc_cd4):
    selection_pool_rrms_cd4 = selection_pool_rrms_cd4[:len(selection_pool_nindc_cd4)]
elif len(selection_pool_rrms_cd4) < len(selection_pool_nindc_cd4):
    selection_pool_nindc_cd4 = selection_pool_nindc_cd4[:len(selection_pool_rrms_cd4)]

all_chunks = selection_pool_rrms_cd4 + selection_pool_nindc_cd4
np.random.shuffle(all_chunks)

X = [selection[0] for selection in all_chunks]
cd4 = [selection[1] for selection in all_chunks]
Y = [selection[2] for selection in all_chunks]

print('prep done')

X_test, X_train, X_valid, y_test, y_train, y_valid = split_test_valid_train(
    X=X,
    y=Y,
    test_perc=test_perc,
    train_perc=train_perc,
    valid_perc=0.5, seed=rand_seed)

prep done


In [21]:
outdir = 'test'
#, nsubset=int(batch_size*len(X_train)/2)
models = []
accs2 = []
for i in range(5):
    model = get_fitted_model(X_train, X_valid, y_train, y_valid,
                             nrun=30, ncell=batch_size, nsubset=int(batch_size*len(X_train)/2),
                             nfilters=[2, 15, 25, 35], max_epochs=100, learning_rate=None,
                             per_sample=False, regression=False,
                             outdir=outdir, verbose=False)
    models.append(model)
    results = model.results
    scaler = results['scaler']
    filters = results['w_best_net']
    #save_results(results, outdir, cytokines)
    test_pred = model.predict(X_test)
    print(f'ROC AUC : {roc_auc_score(y_test, test_pred[:, 1])}')
    print(f'Pred: {test_pred}')
    print(f'Obs: {y_test}')
    test_pred_abs = [1 if pred[0]>pred[1] else 0 for pred in test_pred ]
    acc = accuracy_score(y_test, test_pred_abs)
    print(f'Acc : {acc}')
    accs2.append(acc)

2022-02-25 10:07:21.784177: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-02-25 10:07:24.184733: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb4154eb4f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-02-25 10:07:24.184767: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


57/57 [==============================] - 0s 4ms/step - loss: 0.7002 - accuracy: 0.5000
ROC AUC : 0.0
Pred: [[0.45949541 0.54050457]
 [0.52056026 0.47943973]
 [0.49194232 0.5080577 ]]
Obs: [0, 1, 0]
Acc : 1.0
57/57 [==============================] - 0s 3ms/step - loss: 0.6953 - accuracy: 0.4989

In [22]:
print('Mean Acc: ' + str(sum(accs2)/len(accs2)))

### RESULTS: Also es ist stark abhängig vom Zufall aber auch von der batchsize.
### nsubsets und ncell ist mir noch ein Rätsel


# This is classification. ..
# calculate area under the ROC curve for the test set

Mean Acc: 0.7333333333333332


In [ ]:
#####################################
#################### REGRESSION desease state
#####################################

In [37]:
print('######################################')
print('######################################')
print(' CLASSIFICATION task DONE')
print('######################################')
print('######################################')
print('########### REGRESSION desease state ###########')

model_container = []

X_test, X_train, X_valid, y_test, y_train, y_valid = split_test_valid_train(X=X,
    y=Y,
    test_perc=test_perc,
    train_perc=train_perc,
    valid_perc=0.5, seed=rand_seed)
X = np.array([*X_train, *X_valid])
y = np.array([*y_train, *y_valid])
kf = StratifiedKFold(n_splits=3)
count=1 # this is just there to save the models of all iterations
for train_idx, valid_idx, in kf.split(X=X, y=y):
    outdir = f'ms_pheno_reg_{batch_size}_{count}'
    count = count +1
    X_train, X_valid = X[train_idx], X[valid_idx]
    y_train, y_valid = y[train_idx], y[valid_idx]
    model = get_fitted_model(X_train, X_valid, y_train, y_valid,
                             nsubset=int(batch_size*len(X_train)/2),
                             nfilters=[3, 15, 25, 35], coeff_l1=0,
                             max_epochs=100, nrun=30, learning_rate=None,
                             ncell=batch_size,
                             per_sample=True, regression=True,
                             outdir=outdir_pheno, verbose=False)
    model_container.append(model)
    results = model.results

########### REGRESSION desease state ###########
12/12 [==============================] - 0s 14ms/step - loss: 0.2248


In [39]:
mses = []
for model in model_container:
    print('BS:' + str(batch_size))
    results = model.results
    test_pred = model.predict(X_test)
    mses.append(print_regression_model_stats(test_pred, y_test)[0])

print(f'Mean MSE: {str(sum(mses)/len(mses))}')

# 100 Mean MSE: 0.31166256644638163
# 200 0.36675149071264573

BS:200

Model predictions:
 [0.42304914196332294, 0.5106603304545084, 0.36440641681353253]

True phenotypes:
 [1, 0, 1]
RMSE: 0.3325418228923262
R2: -91.07463179681086
explained_variance_score: -75.91872135903957
BS:200

Model predictions:
 [0.39581475655237836, 0.34935203194618225, 0.2397226244211197]

True phenotypes:
 [1, 0, 1]
RMSE: 0.355036112813966
R2: -81.90470016868397
explained_variance_score: -55.169045185682826
BS:200

Model predictions:
 [0.5032438635826111, 0.6120107968648275, 0.2146938592195511]

True phenotypes:
 [1, 0, 1]
RMSE: 0.412676536431645
R2: -13.68297290989031
explained_variance_score: -11.90805589291468
Mean MSE: 0.36675149071264573
BS:200

Model predictions:
 [0.42304914196332294, 0.5106603304545084, 0.36440641681353253]

True phenotypes:
 [1, 0, 1]
RMSE: 0.3325418228923262
R2: -91.07463179681086
explained_variance_score: -75.91872135903957
BS:200

Model predictions:
 [0.39581475655237836, 0.34935203194618225, 0.2397226244211197]

True phenotypes:
 [1, 0, 1]
R

In [43]:
stats_dict_reg = dict()

for batch_size, values in batch_size_dict.items():

    kf = StratifiedKFold(n_splits=3, random_state=rand_seed)
    model_container = []
    X, Y = values[0], values[2]
    X_test, X_train, X_valid, y_test, y_train, y_valid = split_test_valid_train(X=X,
        y=Y,
        test_perc=test_perc,
        train_perc=train_perc,
        valid_perc=0.5, seed=rand_seed)
    X = np.array([*X_train, *X_valid])
    y = np.array([*y_train, *y_valid])
    for train_idx, valid_idx, in kf.split(X=X, y=y):
        i = 1
        outdir_pheno_reg = f'ms_pheno_reg_{batch_size}_{i}'
        i = i +1
        X_train, X_valid = X[train_idx], X[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]
        model = get_fitted_model(X_train, X_valid, y_train, y_valid,
                                 nsubset=int(batch_size*len(X_train)/2),
                                 nfilters=[3, 15, 25, 35], coeff_l1=0,
                                 max_epochs=100, nrun=30, learning_rate=None,
                                 ncell=batch_size,
                                 per_sample=True, regression=True,
                                 outdir=outdir_pheno_reg, verbose=False)
        model_container.append(model)
        results = model.results

    mses = []
    for model in model_container:
        print('BS:' + str(batch_size))
        results = model.results
        test_pred = model.predict(X_test)
        print_regression_model_stats(test_pred, y_test)
        mse = mean_squared_error(y_test, test_pred)
        print(f'MSE : {mse}')
        mses.append(mse)
        print(f'For: {batch_size}:')
    print(f'Mean MSE: {str(float(sum(mses) / len(mses)))}')
    stats_dict_reg[batch_size] = {'mean_mse': mse}

print('DONE')

# todo meine samples sind gerade abhängig von CD4, das sollte NICHT so sein

/Users/eliasschreiner/.local/share/virtualenvs/CellCnn-O6mYCfgp/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


27/27 [==============================] - 0s 2ms/step - loss: 0.1907
BS:40

Model predictions:
 [0.3975597620010376, 0.5379279951254526, 0.384190171957016, 0.5513476133346558, 0.21399378776550293, 0.49298224846522015, 0.3262913227081299]

True phenotypes:
 [0, 1, 0, 1, 0, 0, 0]
RMSE: 0.15939231715980692
R2: -11.493063446399747
explained_variance_score: -10.185015315240934
MSE : 0.15939231715980692
For: 40:
BS:40

Model predictions:
 [0.3889720141887665, 0.6375779708226522, 0.43673235177993774, 0.4961541493733724, 0.39168309171994525, 0.520780622959137, 0.5046434005101522]

True phenotypes:
 [0, 1, 0, 1, 0, 0, 0]
RMSE: 0.20093397237249278
R2: -30.062365655733018
explained_variance_score: -24.084240243770903
MSE : 0.20093397237249278
For: 40:
BS:40

Model predictions:
 [0.41417185465494794, 0.8839597702026367, 0.5116775433222452, 0.47358909249305725, 0.3009268045425415

/Users/eliasschreiner/.local/share/virtualenvs/CellCnn-O6mYCfgp/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


28/28 [==============================] - 0s 5ms/step - loss: 0.1189
BS:80

Model predictions:
 [0.8342917362848917, 0.14910167207320532, 0.45938289165496826, 0.6156279444694519, 0.6383257706960043]

True phenotypes:
 [1, 0, 1, 0, 1]
RMSE: 0.17035268185363342
R2: -2.259793738175615
explained_variance_score: -2.189395790781922
MSE : 0.17035268185363342
For: 80:
BS:80

Model predictions:
 [0.8480329314867655, 0.36554428935050964, 0.5821284850438436, 0.6239220499992371, 0.6851538817087809]

True phenotypes:
 [1, 0, 1, 0, 1]
RMSE: 0.16394800461586545
R2: -5.695208589620406
explained_variance_score: -5.677274128528786
MSE : 0.16394800461586545
For: 80:
BS:80

Model predictions:
 [0.8902785380681356, 0.09516919404268265, 0.9400432109832764, 0.9342462221781412, 1.166499376296997]

True phenotypes:
 [1, 0, 1, 0, 

/Users/eliasschreiner/.local/share/virtualenvs/CellCnn-O6mYCfgp/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


...

23/23 [==============================] - 0s 3ms/step - loss: 0.2277
BS:150

Model predictions:
 [0.5752330621083578, 0.4521605273087819, 0.8713473280270895]

True phenotypes:
 [0, 1, 1]
RMSE: 0.21585755786230634
R2: -5.974444097400253
explained_variance_score: -5.937633928915517
MSE : 0.21585755786230634
For: 150:
BS:150

Model predictions:
 [0.5227590699990591, 0.5344700515270233, 0.6882615884145101]

True phenotypes:
 [0, 1, 1]
RMSE: 0.19572533848312593
R2: -33.418243900485734
explained_variance_score: -32.15261487965869
MSE : 0.19572533848312593
For: 150:
BS:150

Model predictions:
 [0.5169672767321268, 0.624157726764679, 0.5162976781527201]

True phenotypes:
 [0, 1, 1]
RMSE: 0.21416017190765824
R2: -82.35231625132101
explained_variance_score: -77.27711033626638
MSE : 0.21416017190765824
For: 150:
Mean MSE: 0


/Users/eliasschreiner/.local/share/virtualenvs/CellCnn-O6mYCfgp/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


12/12 [==============================] - 0s 11ms/step - loss: 0.3362
BS:200

Model predictions:
 [0.3754891057809194, 0.5112095177173615, 0.5445660253365835]

True phenotypes:
 [1, 0, 0]
RMSE: 0.3159670613180059
R2: -58.09646260533888
explained_variance_score: -54.23132566513723
MSE : 0.3159670613180059
For: 200:
BS:200

Model predictions:
 [0.46477533380190533, 0.5514942606290182, 0.5396826763947805]

True phenotypes:
 [1, 0, 0]
RMSE: 0.2939562513380809
R2: -198.3555102944264
explained_variance_score: -175.0650510880084
MSE : 0.2939562513380809
For: 200:
BS:200

Model predictions:
 [0.4147348503271739, 0.4991930425167084, 0.4962313175201416]

True phenotypes:
 [1, 0, 0]
RMSE: 0.2793248365354731
R2: -181.38469301249103
explained_variance_score: -169.17959530601942
MSE : 0.2793248365354731
For: 200:
Mean M

/Users/eliasschreiner/.local/share/virtualenvs/CellCnn-O6mYCfgp/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


8/8 [==============================] - 0s 3ms/step - loss: 0.1991
BS:250

Model predictions:
 [0.5893989602724711, 0.6272934675216675]

True phenotypes:
 [0, 0]
RMSE: 0.3704441143828136
R2: -1030.8822976175734
explained_variance_score: 0.0
MSE : 0.3704441143828136
For: 250:
BS:250

Model predictions:
 [0.9492155512173971, 0.47430981198946637]

True phenotypes:
 [0, 0]
RMSE: 0.5629899802112149
R2: -8.984948306336928
explained_variance_score: 0.0
MSE : 0.5629899802112149
For: 250:
BS:250

Model predictions:
 [0.9364844560623169, 0.6277322769165039]

True phenotypes:
 [0, 0]
RMSE: 0.6355254739645559
R2: -25.666922056695498
explained_variance_score: 0.0
MSE : 0.6355254739645559
For: 250:
Mean MSE: 0
DONE


In [44]:
stats_dict_reg


KeyboardInterrupt



In [37]:
batch_sizes = [40, 80, 100,150,200, 250]
batch_size_dict_cd4 = dict()
#batch_size = 20
cluster = 1
for batch_size in batch_sizes:
    ### desease states 1 = RRMS and 0 = NINDC
    selection_pool_rrms_cd4, too_few_data_rrms_cd4 = get_chunks_from_df(rrms_patients2df,
                           freq_df=rrms_patients_freq,
                           desease_state=1,
                           cluster=cluster,
                           batch_size=batch_size)
    selection_pool_nindc_cd4, too_few_data_nindc_cd4 = get_chunks_from_df(nindc_patients2df,
                           freq_df=nindc_patients_freq,
                           desease_state=0,
                           cluster=cluster,
                           batch_size=batch_size)
    #todo make sure list are equally long:
    if len(selection_pool_rrms_cd4) > len(selection_pool_nindc_cd4):
        selection_pool_rrms_cd4 = selection_pool_rrms_cd4[:len(selection_pool_nindc_cd4)]
    elif len(selection_pool_rrms_cd4) < len(selection_pool_nindc_cd4):
        selection_pool_nindc_cd4 = selection_pool_nindc_cd4[:len(selection_pool_rrms_cd4)]

    all_chunks = selection_pool_rrms_cd4 + selection_pool_nindc_cd4
    np.random.shuffle(all_chunks)

    X = [selection[0] for selection in all_chunks]
    cd4 = [selection[1] for selection in all_chunks]
    Y = [selection[2] for selection in all_chunks]

    batch_size_dict_cd4[batch_size] = (X, cd4, Y)

print('prep done')



prep done


In [38]:
from sklearn.model_selection import KFold

### CD4 Freq regression task
model_container_cd4 = []
cluster=1
stats_dict_reg_cd4 = dict()
for batch_size, values in batch_size_dict_cd4.items():
    # for regression task stratified is wrong since there are no classes
    kf = KFold(n_splits=3, random_state=rand_seed)
    model_container = []
    freq_idx =[key for key, value in cluster_to_celltype_dict.items() if key==cluster]
    X, cd4 = values[0], values[1]
    cd4 = [series[freq_idx[0]] for series in cd4]
    X_test, X_train, X_valid, cd4_test, cd4_train, cd4_valid = split_test_valid_train(
        X=X,
        y=cd4,
        test_perc=test_perc,
        train_perc=train_perc,
        valid_perc=0.5, seed=rand_seed)
    X = np.array([*X_train, *X_valid])
    cd4 = np.array([*cd4_train, *cd4_valid])
    for train_idx, valid_idx, in kf.split(X=X):
        i = 1
        outdir_pheno_reg_cd4 = f'ms_pheno_reg_cd4_{batch_size}_{i}'
        i = i +1
        X_train, X_valid = X[train_idx], X[valid_idx]
        cd4_train, cd4_valid = cd4[train_idx], cd4[valid_idx]
        model = get_fitted_model(X_train, X_valid, cd4_train, cd4_valid,
                                 nsubset=int(batch_size*len(X_train)/2),
                                 nfilters=[3, 15, 25, 35], coeff_l1=0,
                                 max_epochs=100, nrun=30, learning_rate=None,
                                 ncell=batch_size,
                                 per_sample=True, regression=True,
                                 outdir=outdir_pheno_reg_cd4, verbose=False)
        model_container_cd4.append(model)
        results = model.results

    mses = []
    for model in model_container_cd4:
        print('BS:' + str(batch_size))
        results = model.results
        test_pred = model.predict(X_test)
        print_regression_model_stats(test_pred, cd4_test)
        mse = mean_squared_error(cd4_test, test_pred)
        print(f'MSE : {mse}')
        mses.append(mse)
        print(f'For: {batch_size}:')
    print(f'Mean MSE: {str(float(sum(mses) / len(mses)))}')
    stats_dict_reg_cd4[batch_size] = {'mean_mse': mse}

print('DONE')

/Users/eliasschreiner/.local/share/virtualenvs/CellCnn-O6mYCfgp/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(
2022-02-25 20:02:16.351606: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-02-25 20:02:16.387002: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fe480832780 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-02-25 20:02:16.387021: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


27/27 [==============================] - 0s 3ms/step - loss: 0.0172
BS:40

Model predictions:
 [0.5347395141919454, 0.537540594736735, 0.5729788939158121, 0.5707543094952902, 0.6353370348612467, 0.5898619691530863, 0.5262790123621622]

True phenotypes:
 [0.5413223140495868, 0.4259927797833935, 0.3575418994413408, 0.3592233009708738, 0.7385398981324278, 0.5642458100558659, 0.45930232558139533]
RMSE: 0.017062515656681994
R2: -12.558694853284717
explained_variance_score: -8.151192868077771
MSE : 0.017062515656681994
For: 40:
BS:40

Model predictions:
 [0.5056720475355784, 0.5433046619097391, 0.5756730238596598, 0.5097556014855703, 0.5626786152521769, 0.6086082259813944, 0.4905570447444916]

True phenotypes:
 [0.5413223140495868, 0.4259927797833935, 0.3575418994413408, 0.3592233009708738, 0.7385398981324278, 0.5642458100558659, 0.45930232558139533]
RMSE: 0.0170208931945

/Users/eliasschreiner/.local/share/virtualenvs/CellCnn-O6mYCfgp/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


28/28 [==============================] - 0s 6ms/step - loss: 0.0218
BS:80

Model predictions:
 [0.6334160566329956, 0.6091871857643127, 0.609617551167806, 0.5812734365463257, 0.6656997203826904]

True phenotypes:
 [0.5431145431145431, 0.5494327390599676, 0.6672354948805461, 0.3840749414519906, 0.6742424242424242]
RMSE: 0.010801001787811154
R2: -12.525652553037819
explained_variance_score: -8.56782618463243
MSE : 0.010801001787811154
For: 80:
BS:80

Model predictions:
 [0.5374995867411295, 0.5408026774724325, 0.5287088553110758, 0.5216595530509949, 0.5471017162005106]

True phenotypes:
 [0.5431145431145431, 0.5494327390599676, 0.6672354948805461, 0.3840749414519906, 0.6742424242424242]
RMSE: 0.010877984111747578
R2: -133.69611630271524
explain

/Users/eliasschreiner/.local/share/virtualenvs/CellCnn-O6mYCfgp/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


...

23/23 [==============================] - 0s 3ms/step - loss: 0.0190
BS:150

Model predictions:
 [0.7214516401290894, 0.6538211703300476, 0.6624873876571655]

True phenotypes:
 [0.7845719661335842, 0.3840749414519906, 0.5439560439560439]
RMSE: 0.030265627662813415
R2: -32.5218391580982
explained_variance_score: -19.51048440064222
MSE : 0.030265627662813415
For: 150:
BS:150

Model predictions:
 [0.6084979573885599, 0.5380025704701742, 0.5472815036773682]

True phenotypes:
 [0.7845719661335842, 0.3840749414519906, 0.5439560439560439]
RMSE: 0.01823561007102019
R2: -17.643377223843814
explained_variance_score: -17.603138566777105
MSE : 0.01823561007102019
For: 150:
BS:150

Model predictions:
 [0.6911916335423788, 0.669826606909434, 0.7178835471471151]

True phenotypes:
 [0.7845719661335842, 0.3840749414519906, 0.5439560439560439]
RMSE: 0.04020822573094228
R2: -103.034858

/Users/eliasschreiner/.local/share/virtualenvs/CellCnn-O6mYCfgp/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


12/12 [==============================] - 0s 3ms/step - loss: 0.0034
BS:200

Model predictions:
 [0.663920263449351, 0.6667243838310242, 0.6898646354675293]

True phenotypes:
 [0.5335968379446641, 0.4527439024390244, 0.7215568862275449]
RMSE: 0.021258746803421116
R2: -156.28516818289367
explained_variance_score: -75.94773778421722
MSE : 0.021258746803421116
For: 200:
BS:200

Model predictions:
 [0.5665313998858134, 0.5429055293401083, 0.5736232002576193]

True phenotypes:
 [0.5335968379446641, 0.4527439024390244, 0.7215568862275449]
RMSE: 0.01036605992678474
R2: -59.11049732596951
explained_variance_score: -58.713022382462526
MSE : 0.01036605992678474
For: 200:
BS:200

Model predictions:
 [0.6833918690681458, 0.6757812897364298, 0.5835704406102499]

True phenotypes:
 [0.5335968379446641, 0.4527439024390244, 0.7215568862275449]
RMSE: 0.030408162218610795
R2: -13.773133064954234
explained_variance_score: -10.795945741469474
MSE : 

/Users/eliasschreiner/.local/share/virtualenvs/CellCnn-O6mYCfgp/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


8/8 [==============================] - 0s 7ms/step - loss: 0.0051
BS:250

Model predictions:
 [0.6793076992034912, 0.6936956246693929]

True phenotypes:
 [0.6672354948805461, 0.5494327390599676]
RMSE: 0.01047875914078654
R2: -201.47597111394134
explained_variance_score: -83.4122201639691
MSE : 0.01047875914078654
For: 250:
BS:250

Model predictions:
 [0.5758403937021891, 0.5678202907244364]

True phenotypes:
 [0.6672354948805461, 0.5494327390599676]
RMSE: 0.004345583287807814
R2: -269.2390964201644
explained_variance_score: -186.3732341064027
MSE : 0.004345583287807814
For: 250:
BS:250

Model predictions:
 [0.6598246097564697, 0.6644019484519958]

True phenotypes:
 [0.6672354948805461, 0.5494327390599676]
RMSE: 0.006636420163275148
R2: -1265.974190284523
explained_variance_score: -713.8179850444011
MSE : 0.006636420163275148
For: 250:
BS:250

Model predictions:
 [0.6033

In [39]:
stats_dict_reg_cd4


{40: {'mean_mse': 0.019643713465870283},
 80: {'mean_mse': 0.009411275954899488},
 100: {'mean_mse': 0.01349146370411641},
 150: {'mean_mse': 0.025294975585536474},
 200: {'mean_mse': 0.027051865979349082},
 250: {'mean_mse': 0.00036837525541905905}}